In [11]:
import cv2, glob, os.path
import tensorflow as tf
from collections import OrderedDict
from operator import itemgetter 
from htmltoolkit 




def optimistic_restore(session, save_file, graph=tf.get_default_graph(), blacklist = []):
    reader = tf.train.NewCheckpointReader(save_file)
    saved_shapes = reader.get_variable_to_shape_map()
    var_names = sorted([(var.name, var.name.split(':')[0]) for var in tf.global_variables()
            if var.name.split(':')[0] in saved_shapes])    
    restore_vars = []
    #skip_names = []
    #restore_names = []
    for var_name, saved_var_name in var_names:   
        valid = True
        for patterns in blacklist:
            #skip_names.append(var_name) 
            valid = valid and saved_var_name.find(patterns) == -1
            
        if valid:
            curr_var = graph.get_tensor_by_name(var_name)
            var_shape = curr_var.get_shape().as_list()
            if var_shape == saved_shapes[saved_var_name]:
                #restore_names.append( saved_var_name)
                restore_vars.append(curr_var)
            else:
                pass
                #skip_names.append(saved_var_name) 
    #print('Skipped:\n',skip_names)
    #print('Restored: \n',restore_names)
    #print('Restored vars:\n',restore_vars)
    opt_saver = tf.train.Saver(restore_vars, allow_empty=True)
    opt_saver.restore(session, save_file)
    


dataset_path = './dataset/new-traffic-sign/'
htk = HtmlToolkit()
js,jQuery = htk.js, htk.jQuery

X_new = []
y_new = []
for path in glob.glob(dataset_path+'*.jpg'):
    X_new.append(cv2.imread(path,cv2.IMREAD_COLOR))
    # exctract the label number from the full path '<some-dir>/<img-num>_<label>.jpg' 
    y_new.append(int(os.path.basename(path).split('.')[0].split('_')[1])) 
n_new = len(X_new)


rows = [[{'content':'', 'attr':{'colspan':2}},'Label','Desc','Match','Predictions']]
for idx in range(n_new):
    num = str(idx)
    img = X_new[idx]
    label = y_new[idx]
    label_num = str(label)
    text = class_texts[label]
    cols = [
        num,
        htk.embedImageArray(img,{'id':'img_'+num}),
        htk.buildTag('div',{'id':'label_'+num},label_num),
        htk.buildTag('div',{'id':'text_'+num},text),
        htk.buildTag('div',{'id':'match_'+num},'-'),
        htk.buildTag('div',{'id':'pred_'+num},'-')
    ]
    rows.append(cols)

htk.printHTML(htk.htmlTable(rows,{'id':'pred_table'}))
X_new = [processImage(image) for image in X_new]
X_new = np.array(X_new)

basedir = './checkpoints/'
restore_name ='20170326-090427' #load old model 
model_name_load = basedir + restore_name+'/model-'+restore_name
check_name_load = basedir + restore_name+'/'
info = loadModelInfo(basedir,restore_name)
hyperparams = info['hyperparams']
#print(hyperparams)

blacklist = ['Adam','_power'];

with tf.variable_scope("lallo",reuse=True):
    with tf.Session() as sess:
            #load weight & baias from the disk
        sess.run(tf.global_variables_initializer())


        x = tf.placeholder(tf.float32, (None, 32, 32, 3))
        y = tf.placeholder(tf.int32, (None))
        one_hot_y = tf.one_hot(y, n_classes)

        logits, modelParams, modelInfo = BuildNet(x,hyperparams)
        model = tf.nn.softmax(logits)
        #cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)

        feed_dict = {}
        for layerName in modelInfo:
            layerInfo = modelInfo[layerName]
            for name in layerInfo['placeholders']:
                if name.endswith('_keep_prob'):
                    plc = layerInfo['placeholders'][name]
                    layerParam = modelParams[layerName]
                    feed_dict[plc] = 1.0
                else:
                    print('Trovato placeholder non gestito :(',name)



        optimistic_restore(sess, model_name_load,blacklist=blacklist)

        #t = modelInfo['0_conv']['variables']['0_conv_W']
        #print(t.name)#,t.eval())
        #tens = tf.get_default_graph().get_tensor_by_name('0_conv_W')
        #print(tens.eval())

        max_bar_length = 100
        predictions = {}
        num_pred = 5
        print(len(X_new))
        for i in range(n_new):
            num = str(i)
            feed_dict.update({ x: X_new[i:i+1], y: y_new[i:i+1] })
            #feed_dict.update({ x: X_new[0:1], y: y_new[0:1] })
            prob = sess.run(model, feed_dict=feed_dict)[0]
            prob_list = []
            for jk in range(len(prob)):
                prob_list.append({
                    'prob':prob[jk],
                    'label': jk
                });
            #print(prob_list)
            prob_list_sorted = sorted(prob_list, key=lambda x: x['prob'], reverse=True)
            #print(prob_list_sorted)

            top5 = prob_list_sorted[:num_pred]
            top5rows = []
            for item in top5:
                prob,label = item['prob'],item['label']
                text = class_texts[label]
                #print(prob,label, text)
                top5rows.append([
                    htk.buildTag('div', { 'class':'prob_bar', 'style':'width: {:.0f}%'.format(prob*max_bar_length) } ) +
                    htk.buildTag('div', {'class':'prob_float'}, "{:.0f}%".format(prob*100) ),
                    str(label),
                    text,
                ])
            table = htk.htmlTable(top5rows, {'id':'pred_table_'+num})
            #print(table)
            print(num)
            jQuery('#pred_'+num).html(table)
print("mmm")




ImportError: cannot import name 'HtmltoolKit'

In [3]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph(modeldir)
new_saver.restore(sess, tf.train.latest_checkpoint(modeldir))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)

NameError: name 'modeldir' is not defined